In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
# Create header (first write to CSV) and template (for consecutive chunk reads) for df
path_chartitems   = 'MIMIC-IV v2.2/icu/d_items.csv'
chartitems        = pd.read_csv(path_chartitems)

template_id     = chartitems.pivot_table(index='label', columns='itemid', aggfunc='size')
template_name   = chartitems.pivot_table(index='label', columns='itemid', aggfunc='size')

In [9]:
# Create template with ids
template_id.columns.name = None
template_id.index.name = None
template_id.insert(0, 'subject_id', 0)
template_id = template_id.iloc[0:0]

In [10]:
# Create template with names and write it to file
for i in template_name.columns:
    template_name.rename(columns={i: chartitems.set_index('itemid').loc[int(i)]['label']}, inplace=True)
template_name.columns.name = None
template_name.index.name = 'subject_id'
template_name = template_name.iloc[0:0]
output_file_path = 'processed_data/chartevents_final.csv'
template_name.to_csv(output_file_path)

In [11]:
# Load data
path_chartevents    = 'MIMIC-IV v2.2/icu/chartevents.csv'
reader              = pd.read_csv(path_chartevents, chunksize=10000000)

In [12]:
i = 0
while True:
    try:
        df_chartevents = next(reader)
    except StopIteration:
        print("No more chunks left.")
        break

    if i == 50:
        break

    # Keep most recent chart values for each subject
    df = df_chartevents.sort_values(by=['subject_id', 'itemid', 'charttime'], ascending=[True, True, False])
    df.drop_duplicates(subset=['subject_id', 'itemid'], keep='first', inplace=True)

    # Drop irrelevant columns
    df = df[['subject_id', 'itemid', 'valuenum']]

    # Reformat table
    df.reset_index(drop=True, inplace=True)
    df = df.pivot_table(index='subject_id', columns='itemid', values='valuenum', aggfunc='mean')
    df.insert(0, 'subject_id', df.index)
    df.reset_index(drop=True, inplace=True)
    
    # Update df formatting to match header
    missing_columns = set(template_id.columns) - set(df.columns)
    new_columns = {col: pd.Series([pd.NA] * len(df), name=col) for col in missing_columns}
    new_columns_df = pd.concat(new_columns.values(), axis=1)
    df = pd.concat([df, new_columns_df], axis=1)

    df.to_csv(output_file_path, mode='a', header=False, index=False)

    i += 1

    print(i)

1
2
3
4
5
6
7
8
9


KeyboardInterrupt: 